### We import libraries

In [1]:
import json
import pickle
import zipfile

!pip install tf_slim

In [2]:
import pandas as pd

### We unzip the original database

In [3]:
local_zip = "/content/Aquatic_Animals.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("dataset_original")
zip_ref.close()

## JSON to CSV
### We define the path of our JSON file
This process must be repeated with training and test

In [7]:
type_file = "test"
path = "/content/AquaticAnimals.json"
data_file = open(path)
data = json.load(data_file)

In [ ]:
data

We define the structure of the source JSON and the relevant variables that we must save in the CSV

In [8]:
csv_list = []
'''
Recorremos la estructura del archivo json, extrayendo las variables relevantes
1. Clase
2. Bounding box (x inicial, y inicial, x final, y final)
3. Ancho y alto de la imagen.
4. Nombre del archivo.
'''
for classification in data:
  width, height = classification['width'], classification['height']
  image = classification['image']
  for item in classification['tags']:
    name = item['name']
    xmin = item['pos']['x']
    ymin = item['pos']['y']
    xmax = item['pos']['x'] + item['pos']['w']
    ymax = item['pos']['y'] + item['pos']['h']

    value = (image, width, height, name, xmin, ymin, xmax, ymax)
    csv_list.append(value)

column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
csv_df = pd.DataFrame(csv_list, columns = column_name)

# We convert the dataframe to CSV
csv_df.to_csv("/content/{}_labels.csv".format(type_file))

## CSV to TFRecord
We install the TensorFlow Object Detection library

In [9]:
import os
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get update && apt-get install -y -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

/content
/content/models
Note: switching to '58d19c67e1d30d905dd5c6e5092348658fed80af'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 58d19c67e Internal change
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-secur

We take the base script and modify it to our use case

In [11]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf
import sys
sys.path.append("../../models/research")

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict


# It is essential to replace the classes with the same classes as the project.
# The names must be spelled exactly the same.
# If there are more classes in your project you would add an ELIF.
def class_text_to_int(row_label):
    if row_label == "Blacktip shark":
      return 1
    elif row_label == "Bull shark":
      return 2
    elif row_label == "Hammerhead shark":
      return 3
    elif row_label == "Lemon shark":
      return 4
    elif row_label == "Nurse shark":
      return 5
    elif row_label == "Tiger shark":
      return 6
    elif row_label == "Whitetip shark":
      return 7
    elif row_label == "Caribbean giant manta ray":
      return 8
    elif row_label == "Chilean devil ray":
      return 9
    elif row_label == "Spotted eagle ray":
      return 10
    elif row_label == "Ocellated eagle ray":
      return 11
    elif row_label == "Green sea turtle":
      return 12
    elif row_label == "Hawksbill sea turtle":
      return 13
    elif row_label == "Kemp's ridley sea turtle":
      return 14
    elif row_label == "Leatherback sea turtle":
      return 15
    elif row_label == "Olive ridley sea turtle":
      return 16
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

# We modify with the location of our files. Remember to take the time
# to check that the URL is the same as the Google Colab where you uploaded the
# file manually
output_path = "test.record"
image_dir = "/content/dataset_original/Aquatic_Animals"
csv_input = "/content/test_labels.csv"

writer = tf.io.TFRecordWriter(output_path)
path = os.path.join(image_dir)
examples = pd.read_csv(csv_input)
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())

writer.close()
output_path = os.path.join(os.getcwd(), output_path)
print('Successfully created the TFRecords: {}'.format(output_path))

Successfully created the TFRecords: /content/models/research/test.record
